[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pablo-sampaio/rl_facil/blob/main/cap08/cap08-main.ipynb)

# Capítulo 8 - DQN


## 1 - Configurações Iniciais

Instalação de pacotes e atribuição do caminho para o código do projeto.

In [1]:
import pygame # this import here is just to prevent a strange bug (with some dynamic library used by pygame and other packages)

import sys
from IPython.display import clear_output

IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    !pip install opencv-python
    !pip install swig
    !pip install gymnasium[all]   # roda no 1.0.0
    !pip install ale-py           # para os jogos de Atari
    #!pip install autorom[accept-rom-license]
    !pip install tensorboard

    !git clone https://github.com/pablo-sampaio/rl_facil
    sys.path.append("/content/rl_facil")

else:
    from os import path
    sys.path.append( path.dirname( path.dirname( path.abspath("__main__") ) ) )

clear_output()

Carrega a extensão para visualizar o `tensorboard` em um notebook.

In [2]:
%load_ext tensorboard

Importa pacotes.

In [3]:
import time
from datetime import datetime
import collections

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

from torch.utils.tensorboard import SummaryWriter
import tensorboard

import matplotlib.pyplot as plt

In [4]:
import gymnasium as gym
import ale_py

In [5]:
from cap08 import dqn_models
from cap08.atari_wrappers import *
from cap08.qnet_helper import record_video_qnet

from util.notebook import display_videos_from_path, display_videos_from_path_widgets

In [6]:
# usar GPU compatível com CUDA costuma ser mais rápido
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

torch.set_default_device(device)

## 2 - DQN - Definições Auxiliares

Adaptado do código explicado no livro **"Deep Reinforcement Learning Hands On"** (Maxim Lapan), Chapter 6.

### Classes Auxiliares

In [7]:
Experience = collections.namedtuple('Experience', field_names=['state', 'action', 'reward', 'done', 'new_state'])

class DQNExperienceBuffer:
    def __init__(self, capacity):
        self.buffer = collections.deque(maxlen=capacity)

    def __len__(self):
        return len(self.buffer)

    def append(self, s1, a, r, done, s2):
        experience = Experience(s1, a, r, done, s2)
        self.buffer.append(experience)

    def sample(self, batch_size):
        indices = np.random.choice(len(self.buffer), batch_size, replace=False)
        states, actions, rewards, dones, next_states = zip(*[self.buffer[idx] for idx in indices])
        return np.array(states), np.array(actions), np.array(rewards, dtype=np.float32), \
               np.array(dones, dtype=np.uint8), np.array(next_states)

### Funções Auxiliares

In [8]:
# Faz uma escolha epsilon-greedy
def epsilon_greedy_qnet(qnet, env, state, epsilon):
    if np.random.random() < epsilon:
        action = env.action_space.sample()
    else:
        state_v = torch.tensor(state, dtype=torch.float32)
        state_v = state_v.unsqueeze(0)  # Adiciona dimensão de batch como eixo 0 (e.g. transforma uma lista [a,b,c] em [[a,b,c]])
        q_vals_v = qnet(state_v)
        _, act_v = torch.max(q_vals_v, dim=1)
        action = int(act_v.item())
    return action

In [9]:
# loss function, para treinamento da rede no DQN
def calc_loss(batch, net, tgt_net, gamma):
    states, actions, rewards, dones, next_states = batch

    states_v = torch.tensor(states, dtype=torch.float32)
    next_states_v = torch.tensor(next_states, dtype=torch.float32)
    actions_v = torch.tensor(actions, dtype=torch.int64)
    rewards_v = torch.tensor(rewards)
    done_mask = torch.tensor(dones, dtype=torch.bool)

    state_action_values = net(states_v).gather(1, actions_v.unsqueeze(-1)).squeeze(-1)
    next_state_values = tgt_net(next_states_v).max(dim=1)[0]
    next_state_values[done_mask] = 0.0
    next_state_values = next_state_values.detach()

    target_state_action_values = rewards_v + gamma * next_state_values
    return nn.MSELoss()(state_action_values, target_state_action_values)

## 3 - DQN - Função Principal

Esta é a função que faz o aprendizado. (Porém, o DQN é uma solução maior, pensada para jogos de Atari, e que inclui também os wrappers, que serão usados na seção 6.)

In [10]:
def DQN_TRAIN(env, env_name, gamma, qnet, qnet_lr, target_qnet, target_update_freq, replay_size, batch_size, epsilon_f, epsilon_decay_period, GOAL_REWARD):
    print(qnet)

    # Cria o otimizador, que vai fazer o ajuste dos pesos da 'qnet',
    # Usa uma técnica de gradiente descendente de destaque, chamada ADAM
    optimizer = optim.Adam(qnet.parameters(), lr=qnet_lr)

    # Para o logging de dados, para serem exibidos no tensorboard
    writer = SummaryWriter(comment="-" + env_name)

    buffer = DQNExperienceBuffer(replay_size)

    start_time_str = datetime.now().strftime("%Y-%m-%d,%H-%M-%S")
    episode_reward_list = []
    step = 0
    epsilon = 1.0

    state, _ = env.reset()
    episode_reward = 0.0
    episode_start_step = 0
    episode_start_time = time.time()

    while True:
        # Decaimento linear do epsilon
        epsilon = max(epsilon_f, 1.0 - step / epsilon_decay_period)

        action = epsilon_greedy_qnet(qnet, env, state, epsilon)

        # Faz um passo / Aplica uma ação no ambiente
        new_state, reward, terminated, truncated, _ = env.step(action)

        step += 1
        done = terminated or truncated
        episode_reward += reward

        # Adiciona no buffer
        buffer.append(state, action, reward, terminated, new_state)
        state = new_state

        if step % 10_000 == 0:
            clear_output()

        if done:
            episode_reward_list.append(episode_reward)
            speed = (step - episode_start_step) / (time.time() - episode_start_time + 0.00001)

            state, _ = env.reset()
            episode_reward = 0.0
            episode_start_step = step
            episode_start_time = time.time()

            # Abaixo, faz vários loggings de dados
            mean_reward = np.mean(episode_reward_list[-100:])
            print(f"{step}: finished {len(episode_reward_list)} episodes, mean reward {mean_reward:.3f}, eps {epsilon:.2f}, speed {speed:.2f} steps/s")
            writer.add_scalar("epsilon", epsilon, step)
            writer.add_scalar("epi_reward_100", mean_reward, step)
            writer.add_scalar("epi_reward", episode_reward, step)

            # Testa se "resolveu" o ambiente
            if mean_reward > GOAL_REWARD:
                print(f"Solved in {step} steps with mean reward {mean_reward:.3f}")
                filename = env_name + "-" + start_time_str + ".dat"
                torch.save(qnet.state_dict(), filename)
                print(f"Model saved as {filename}")
                break

        if len(buffer) >= replay_size:
            # Faz a 'tgt_net' receber os mesmos valores de pesos da 'qnet', na frequência indicada
            if step % target_update_freq == 0:
                target_qnet.load_state_dict(qnet.state_dict())

            # Escolhe amostras aleatórios do buffer e faz uma atualização dos pesos da rede
            optimizer.zero_grad()
            batch = buffer.sample(batch_size)
            loss_t = calc_loss(batch, qnet, target_qnet, gamma)
            loss_t.backward()
            optimizer.step()

    writer.close()

## 4 - Treinando em um Ambientes Simples

In [11]:
SIMPLE_ENV_NAME = "CartPole-v1"
GOAL_REWARD = 200

#SIMPLE_ENV_NAME = "MountainCar-v0"
#GOAL_REWARD = -120

GAMMA = 0.999
REPLAY_SIZE = 5_000
BATCH_SIZE = 32
LEARNING_RATE = 0.001
SYNC_TARGET_FRAMES = 1_000

EPSILON_DECAY_PERIOD = 40_000
EPSILON_FINAL = 0.02

In [12]:
# Cria o ambiente
env1 = gym.make(SIMPLE_ENV_NAME)

# Cria as redes neurais
qnet1 = dqn_models.MLP(env1.observation_space.shape[0], [128,128], env1.action_space.n)
qtarget1 = dqn_models.MLP(env1.observation_space.shape[0], [128,128], env1.action_space.n)

In [13]:
# Para carregar uma rede salva de arquivo, descomente o bloco abaixo
# Essa rede salva pode ser testada ou treinada por mais alguns passos
'''
filename = filename + "<Nome do arquivo>.dat"
qnet1.load_state_dict(torch.load(filename, map_location=lambda storage,loc: storage))
#''';

In [ ]:
# Para treinar o agente, rode o código abaixo
#'''

DQN_TRAIN(
    env = env1,
    env_name = SIMPLE_ENV_NAME,
    gamma = GAMMA,
    qnet = qnet1,
    qnet_lr = LEARNING_RATE,
    target_qnet = qtarget1,
    target_update_freq = SYNC_TARGET_FRAMES,
    replay_size = REPLAY_SIZE,
    batch_size = BATCH_SIZE,
    epsilon_f = EPSILON_FINAL,
    epsilon_decay_period = EPSILON_DECAY_PERIOD,
    GOAL_REWARD = GOAL_REWARD)

#'''

In [ ]:
record_video_qnet(SIMPLE_ENV_NAME, qnet1, episodes=2, folder="./dqn-simple")

In [ ]:
display_videos_from_path('./dqn-simple')

## 5 - Visualização dos Resultados dos Treinamentos

Estamos, aqui, usando o **Tensorboard** para acompanhar os dados do treinamento em tempo real. Este módulo foi criado para o Tensorflow, mas é compatível com Pytorch.

Basta rodar uma vez e acompanhar. Ele pode também ser executado antes da seção 4, para acompanhar.


In [ ]:
if IN_COLAB:
    %tensorboard --logdir runs
else:
    %tensorboard --logdir cap08/runs

In [18]:
# Mostra o tensorboard no notebook, ocupando a altura indicada
#tensorboard.notebook.display(height=500)

## 6 - Treinando no Jogo Pong (Atari)

In [19]:
# Veja outros em: https://gymnasium.farama.org/environments/atari/
# Se mudar o jogo, lembre-se de alterar também o GOAL_REWARD abaixo!
ATARI_ENV_NAME = "PongNoFrameskip-v4"

# Recompensa alvo; no Pong, esta é a diferença de pontos do player para a "cpu", sendo +21.0 o máximo e -21.0 o mínimo
# Tente com algum valor negativo (e.g. -15.0) para um treinamento mais rápido, ou algum valor positivo (+15.0) para ver o agent ganhar da "cpu"
GOAL_REWARD = -10.0 #0.0

# Parâmetros do DQN
GAMMA = 0.99
BATCH_SIZE = 32
REPLAY_SIZE = 20_000
LEARNING_RATE = 1e-4
SYNC_TARGET_FRAMES = 1_500

EPSILON_DECAY_PERIOD = 100_000
EPSILON_FINAL = 0.02

env2 = gym.make(ATARI_ENV_NAME)

In [20]:
# Aplica os wrappers do DQN para ambientes Atari
env2a = MaxAndSkipEnv(env2)
env2b = FireResetEnv(env2a)
env2c = ProcessFrame84(env2b)
env2d = ImageToPyTorch(env2c)
env2e = BufferWrapper(env2d, 4)
env2f = ScaledFloatFrame(env2e)

# Cria as redes neurais
qnet2 = dqn_models.DQNNet(env2f.observation_space.shape, env2f.action_space.n)
qtarget2 = dqn_models.DQNNet(env2f.observation_space.shape, env2f.action_space.n)

In [ ]:
# Mostrando a tela do ambiente original, e outras telas após passar por alguns wrappers
f, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(12,5))

s, _ = env2.reset()
ax1.imshow(s)
ax1.set_title('Tela antes de iniciar')

s, _ = env2b.reset()
ax2.imshow(s)
ax2.set_title('Tela da partida iniciada automaticamente')

s, _ = env2d.reset()
ax3.imshow(s[0], cmap='gray', vmin=0, vmax=255)  # exibe em escala de cinza
ax3.set_title('Imagem processada');

In [22]:
# Para carregar uma rede salva de arquivo, descomente o bloco abaixo
# Permite testar ou treinar mais
'''
filename = "/content/rl_facil/cap08/" if IN_COLAB else ""
filename = filename + "PongNoFrameskip-v4-2023-11-21,18-35-12.dat"
qnet2.load_state_dict(torch.load(filename, map_location=lambda storage,loc: storage))
#''';

In [ ]:
# Para treinar o agente, rode o código abaixo

DQN_TRAIN(
    env = env2f,
    env_name = ATARI_ENV_NAME,
    gamma = GAMMA,
    qnet = qnet2,
    qnet_lr = LEARNING_RATE,
    target_qnet = qtarget2,
    target_update_freq = SYNC_TARGET_FRAMES,
    replay_size = REPLAY_SIZE,
    batch_size = BATCH_SIZE,
    epsilon_f = EPSILON_FINAL,
    epsilon_decay_period = EPSILON_DECAY_PERIOD,
    GOAL_REWARD = GOAL_REWARD)


In [ ]:
# Roda alguns episódigos com o modelo e salva os vídeos em arquivos
# Atenção: precisa rodar com os wrappers aplicados!
env2 = gym.make(ATARI_ENV_NAME, render_mode="rgb_array")
env2a = MaxAndSkipEnv(env2)
env2b = FireResetEnv(env2a)
env2c = ProcessFrame84(env2b)
env2d = ImageToPyTorch(env2c)
env2e = BufferWrapper(env2d, 4)
env2f = ScaledFloatFrame(env2e)

record_video_qnet(env2f, qnet2, episodes=1, folder="./dqn-atari")

In [ ]:
display_videos_from_path('./dqn-atari')
#display_videos_from_path_widgets("./dqn-atari")